https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2025/10-kubernetes/homework.md

In this homework, we'll deploy the lead scoring model from the homework 5.

## Building the image

`docker build -f Dockerfile_full -t zoomcamp-model:3.13.10-hw10 .`

## Question 1

Run it to test that it's working locally:

docker run -it --rm -p 9696:9696 zoomcamp-model:3.13.10-hw10

In [3]:
!python q6_test.py

{'conversion_probability': 0.49999999999842815, 'conversion': False}


Ans: the probability of getting a subscription is 0.49

## Installing `kubectl` and `kind`

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/

In [5]:
!kubectl version --client

Client Version: v1.34.3
Kustomize Version: v5.7.1


What's the version of kind that you have?

In [6]:
!kind --version

kind version 0.30.0


Ans: 0.30.0

## Creating a cluster

Now let's create a cluster with `kind`:

In [7]:
!kind create cluster

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.34.0) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a question, bug, or feature request? Let us know! https://kind.sigs.k8s.io/#community 🙂


And check with `kubectl` that it was successfully created:

In [8]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:43665
CoreDNS is running at https://127.0.0.1:43665/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Question 3

What's the smallest deployable computing unit that we can create and manage in Kubernetes (`kind` in our case)?

Ans: In Kubernetes, the smallest deployable and manageable unit is a __Pod__.

## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?

In [9]:
!kubectl get svc

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   21m


Ans: ClusterIP

## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:3.13.10-hw10`), we need to register it with `kind`.

What's the command we need to run for that?

In [11]:
!kind load docker-image zoomcamp-model:3.13.10-hw10

Image: "zoomcamp-model:3.13.10-hw10" with ID "sha256:81a8079a43fc36e014193b389ea7afa39c710ddc038b1af9d261e54d5703b91d" not yet present on node "kind-control-plane", loading...


Ans: kind load docker-image

## Question 6

Now let's create a deployment config: [deployment.yaml](./k8s/deployment.yaml)

What is the value for `Port`?

Ans: 9696

Apply this deployment using the appropriate command and get a list of running Pods. You can see one running Pod.

In [3]:
!kubectl apply -f k8s/deployment.yaml

deployment.apps/subscription created


In [4]:
!kubectl get deployments

NAME           READY   UP-TO-DATE   AVAILABLE   AGE
subscription   1/1     1            1           22s


In [5]:
!kubectl get pods

NAME                            READY   STATUS    RESTARTS   AGE
subscription-69c87b4597-nc8z7   1/1     Running   0          32s


In [7]:
!kubectl describe deployment subscription

Name:                   subscription
Namespace:              default
CreationTimestamp:      Wed, 17 Dec 2025 19:52:18 +0000
Labels:                 <none>
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=subscription
Replicas:               1 desired | 1 updated | 1 total | 1 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=subscription
  Containers:
   subscription:
    Image:      zoomcamp-model:3.13.10-hw10
    Port:       9696/TCP
    Host Port:  0/TCP
    Limits:
      cpu:     500m
      memory:  128Mi
    Requests:
      cpu:         100m
      memory:      64Mi
    Environment:   <none>
    Mounts:        <none>
  Volumes:         <none>
  Node-Selectors:  <none>
  Tolerations:     <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailabl

## Question 7

Let's create a service for this deployment: [service.yaml](./k8s/service.yaml)

Answer: `subscription` is the app selector

In [8]:
!kubectl apply -f k8s/service.yaml

service/subscription created


In [9]:
!kubectl get services

NAME           TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
kubernetes     ClusterIP      10.96.0.1      <none>        443/TCP        10h
subscription   LoadBalancer   10.96.79.131   <pending>     80:30660/TCP   11s


In [10]:
!kubectl describe service subscription

Name:                     subscription
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=subscription
Type:                     LoadBalancer
IP Family Policy:         SingleStack
IP Families:              IPv4
IP:                       10.96.79.131
IPs:                      10.96.79.131
Port:                     <unset>  80/TCP
TargetPort:               9696/TCP
NodePort:                 <unset>  30660/TCP
Endpoints:                10.244.0.5:9696
Session Affinity:         None
External Traffic Policy:  Cluster
Internal Traffic Policy:  Cluster
Events:                   <none>


## Testing the service

We can test our service locally by forwarding the port 9696 on our computer to the port 80 on the service:

`kubectl port-forward service/subscription 9696:80`

Run q6_test.py once again to verify that everything is working. You should get the same result as in Question 1.

In [13]:
!python q6_test.py

{'conversion_probability': 0.49999999999842815, 'conversion': False}


## Autoscaling

Now we're going to use a [HorizontalPodAutoscaler](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/) 
(HPA for short) that automatically updates a workload resource (such as our deployment), 
with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

In [14]:
!kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3

Flag --cpu-percent has been deprecated, Use --cpu with percentage or resource quantity format (e.g., '70%' for utilization or '500m' for milliCPU).
horizontalpodautoscaler.autoscaling/subscription-hpa autoscaled


You can check the current status of the new HPA by running:

In [15]:
!kubectl get hpa

NAME               REFERENCE                 TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: <unknown>/20%   1         3         0          14s


In [17]:
!kubectl get hpa

NAME               REFERENCE                 TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: <unknown>/20%   1         3         1          83s


## Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing
`q6_test.py` script by putting the operator that sends the request to the subscription service into a loop.

See: [q6_test.py](q6_test.py) and run this script

## Question 8 (optional)

Run `kubectl get hpa subscription-hpa --watch` command to monitor how the autoscaler performs. 
Within a minute or so, you should see the higher CPU load; and then - more replicas. 
What was the maximum amount of the replicas during this test?

In [24]:
!kubectl get hpa subscription-hpa

NAME               REFERENCE                 TARGETS        MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: 14%/20%   1         3         2          17m
